# 客户评论分类

In [1]:
#创建词汇表
import numpy as np
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],                #切分的词条
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]                                                         #类别标签向量，1代表侮辱性词汇，0代表不是
    return postingList,classVec
 
if __name__ == '__main__':
    postingLIst, classVec = loadDataSet()
    for each in postingLIst:
        print(each)
    print(classVec)

['my', 'dog', 'has', 'flea', 'problems', 'help', 'please']
['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid']
['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him']
['stop', 'posting', 'stupid', 'worthless', 'garbage']
['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him']
['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']
[0, 1, 0, 1, 0, 1]


In [2]:
#词条转换为词条向量
def createVocabList(dataSet):
    vocabSet = set()                      #创建一个空的不重复列表
    for document in dataSet:               
        vocabSet = vocabSet | set(document) #取并集
    return list(vocabSet)


def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)                                    #创建一个其中所含元素都为0的向量
    for word in inputSet:                                                #遍历每个词条
        if word in vocabList:                                            #如果词条存在于词汇表中，则置1
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec                                                    #返回文档向量
 
if __name__ == '__main__':
    postingList, classVec = loadDataSet()
    print('postingList:\n',postingList)
    myVocabList = createVocabList(postingList)#词汇表，所有单词出现的集合
    print('myVocabList:\n',myVocabList)
    trainMat = []
    for postinDoc in postingList:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))#词条向量化
    print('trainMat:\n', trainMat)

postingList:
 [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'], ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'], ['stop', 'posting', 'stupid', 'worthless', 'garbage'], ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
myVocabList:
 ['food', 'not', 'stupid', 'dog', 'I', 'ate', 'licks', 'buying', 'quit', 'please', 'mr', 'dalmation', 'my', 'love', 'him', 'take', 'problems', 'help', 'posting', 'garbage', 'to', 'steak', 'worthless', 'stop', 'how', 'has', 'maybe', 'flea', 'so', 'cute', 'is', 'park']
trainMat:
 [[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [13]:
#训练朴素贝叶斯分类器
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)   #m                         #计算训练的文档数目
    numWords = len(trainMatrix[0])#32  #n    #                   #计算每篇文档的词条数
    #trainCategory = [0,1,0,1,0,1]   
    pAbusive = sum(trainCategory)/float(numTrainDocs) #P(H1)       #文档属于侮辱类的概率，先验概率
    p0Num = np.zeros(numWords); p1Num = np.zeros(numWords)    #创建numpy.zeros数组,词条出现数初始化为0
    p0Denom = 0.0; p1Denom = 0.0                            #分母初始化为0
    for i in range(numTrainDocs):    #文本数，样本数
        if trainCategory[i] == 1:               #统计属于侮辱类的条件概率所需的数据，即P(w0|1),P(w1|1),P(w2|1)···
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:                                   #统计属于非侮辱类的条件概率所需的数据，即P(w0|0),P(w1|0),P(w2|0)···
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = p1Num/p1Denom                                      
    p0Vect = p0Num/p0Denom         
    return p0Vect,p1Vect,pAbusive           #返回属于侮辱类的条件概率数组，属于非侮辱类的条件概率数组，文档属于侮辱类的概率
 
if __name__ == '__main__':
    postingList, classVec = loadDataSet()
    myVocabList = createVocabList(postingList)
    print('myVocabList:\n', myVocabList)
    trainMat = []
    for postinDoc in postingList:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V, p1V, pAb = trainNB0(trainMat, classVec)
    print('p0V:\n', p0V)
    print('p1V:\n', p1V)
    print('classVec:\n', classVec)
    print('pAb:\n', pAb)

myVocabList:
 ['love', 'help', 'quit', 'has', 'worthless', 'I', 'maybe', 'please', 'is', 'flea', 'problems', 'him', 'dog', 'steak', 'not', 'park', 'stop', 'take', 'to', 'so', 'garbage', 'my', 'mr', 'cute', 'how', 'food', 'ate', 'buying', 'licks', 'posting', 'dalmation', 'stupid']
p0V:
 [0.04166667 0.04166667 0.         0.04166667 0.         0.04166667
 0.         0.04166667 0.04166667 0.04166667 0.04166667 0.08333333
 0.04166667 0.04166667 0.         0.         0.04166667 0.
 0.04166667 0.04166667 0.         0.125      0.04166667 0.04166667
 0.04166667 0.         0.04166667 0.         0.04166667 0.
 0.04166667 0.        ]
p1V:
 [0.         0.         0.05263158 0.         0.10526316 0.
 0.05263158 0.         0.         0.         0.         0.05263158
 0.10526316 0.         0.05263158 0.05263158 0.05263158 0.05263158
 0.05263158 0.         0.05263158 0.         0.         0.
 0.         0.05263158 0.         0.05263158 0.         0.05263158
 0.         0.15789474]
classVec:
 [0, 1, 0, 

# 算法存在的问题
1.利用贝叶斯分类器对文档进行分类时，要计算多个概率的乘积以获得文档属于某个类别的概率，即计算p(w0|1)p(w1|1)p(w2|1)。如果其中有一个概率值为0，那么最后的成绩也为0。
为了降低这种影响，可以将所有词的出现数初始化为1，并将分母初始化为2。这种做法就叫做拉普拉斯平滑(Laplace Smoothing)又被称为加1平滑，是比较常用的平滑方法，它就是为了解决0概率问题
2.下溢出问题：太多很小的数相乘，导致下溢出，结果可能变成零。
为了解决这个问题，对乘积结果取自然对数。通过求对数可以避免下溢出或者浮点数舍入导致的错误

In [14]:
#修改后的算法
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)                            #计算训练的文档数目
    numWords = len(trainMatrix[0])                            #计算每篇文档的词条数
    pAbusive = sum(trainCategory)/float(numTrainDocs)        #文档属于侮辱类的概率
    p0Num = np.ones(numWords); p1Num = np.ones(numWords)    #创建numpy.ones数组,词条出现数初始化为1，拉普拉斯平滑
    p0Denom = 2.0; p1Denom = 2.0                            #分母初始化为2,拉普拉斯平滑
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:                            #统计属于侮辱类的条件概率所需的数据，即P(w0|1),P(w1|1),P(w2|1)···
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:                                                #统计属于非侮辱类的条件概率所需的数据，即P(w0|0),P(w1|0),P(w2|0)···
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom)                            #取对数，防止下溢出         
    p0Vect = np.log(p0Num/p0Denom)         
    return p0Vect,p1Vect,pAbusive                            #返回属于侮辱类的条件概率数组，属于非侮辱类的条件概率数组，文档属于侮辱类的概率

In [15]:
#分类函数
"""
函数说明:朴素贝叶斯分类器分类函数
 
Parameters:
    vec2Classify - 待分类的词条数组
    p0Vec - 侮辱类的条件概率数组
    p1Vec -非侮辱类的条件概率数组
    pClass1 - 文档属于侮辱类的概率
Returns:
    0 - 属于非侮辱类
    1 - 属于侮辱类
"""
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)        #对应元素相乘。logA * B = logA + logB，所以这里加上log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

In [18]:
#测试
def testingNB():
    listOPosts,listClasses = loadDataSet()                                    #创建实验样本
    myVocabList = createVocabList(listOPosts)                                #创建词汇表
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))                #将实验样本向量化
    p0V,p1V,pAb = trainNB0(np.array(trainMat),np.array(listClasses))        #训练朴素贝叶斯分类器
    testEntry = ['love', 'my', 'dalmation']                                    #测试样本1
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry)) 
    print(thisDoc)
    #测试样本向量化
    if classifyNB(thisDoc,p0V,p1V,pAb):
        print(testEntry,'属于侮辱类')                                        #执行分类并打印分类结果
    else:
        print(testEntry,'属于非侮辱类')                                        #执行分类并打印分类结果
        
    testEntry = ['stupid', 'garbage']                                        #测试样本2
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))                #测试样本向量化
    if classifyNB(thisDoc,p0V,p1V,pAb):
        print(testEntry,'属于侮辱类')                                        #执行分类并打印分类结果
    else:
        print(testEntry,'属于非侮辱类')                                        #执行分类并打印分类结 
if __name__ == '__main__':
    testingNB()

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0]
['love', 'my', 'dalmation'] 属于非侮辱类
['stupid', 'garbage'] 属于侮辱类


# 词袋模型向量化

In [1]:
from sklearn.feature_extraction.text import CountVectorizer  
vectorizer=CountVectorizer()
corpus=["I come to China to travel", 
    "This is a car polupar in China",          
    "I love tea and Apple ",   
    "The work is to write some papers in science"] 
print(vectorizer.fit_transform(corpus))

  (0, 16)	1
  (0, 3)	1
  (0, 15)	2
  (0, 4)	1
  (1, 5)	1
  (1, 9)	1
  (1, 2)	1
  (1, 6)	1
  (1, 14)	1
  (1, 3)	1
  (2, 1)	1
  (2, 0)	1
  (2, 12)	1
  (2, 7)	1
  (3, 10)	1
  (3, 8)	1
  (3, 11)	1
  (3, 18)	1
  (3, 17)	1
  (3, 13)	1
  (3, 5)	1
  (3, 6)	1
  (3, 15)	1


In [5]:
print(vectorizer.fit_transform(corpus).toarray())
print(vectorizer.get_feature_names())
#"I"在英文中是停用词，不参加词频的统计，由于大部分的文本都只会使用词汇表中的很少一部分的词，
#因此我们的词向量中会有大量的0。也就是说词向量是稀疏的。在实际应用中一般使用稀疏矩阵来存储。

[[0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 2 1 0 0]
 [0 0 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0]
 [1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1]]
['and', 'apple', 'car', 'china', 'come', 'in', 'is', 'love', 'papers', 'polupar', 'science', 'some', 'tea', 'the', 'this', 'to', 'travel', 'work', 'write']


# TF-IDF模型

In [21]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
corpus=["I come to China travel travel travel travel travel", 
    "This is a car to polupar in China",          
    "I love tea and to Apple ",   
    "The work is to write some papers in science"] 

tfidf = TfidfVectorizer()
re = tfidf.fit_transform(corpus)
print(re)

  (0, 4)	0.19282929643440955
  (0, 15)	0.10062632628078319
  (0, 3)	0.15202883217044438
  (0, 16)	0.9641464821720478
  (1, 15)	0.23023944874847488
  (1, 3)	0.3478516587709524
  (1, 14)	0.4412057217485169
  (1, 6)	0.3478516587709524
  (1, 2)	0.4412057217485169
  (1, 9)	0.4412057217485169
  (1, 5)	0.3478516587709524
  (2, 15)	0.25246826075544676
  (2, 7)	0.4838025881780501
  (2, 12)	0.4838025881780501
  (2, 0)	0.4838025881780501
  (2, 1)	0.4838025881780501
  (3, 15)	0.19035292263071363
  (3, 6)	0.2875900730692273
  (3, 5)	0.2875900730692273
  (3, 13)	0.36477154142239465
  (3, 17)	0.36477154142239465
  (3, 18)	0.36477154142239465
  (3, 11)	0.36477154142239465
  (3, 8)	0.36477154142239465
  (3, 10)	0.36477154142239465


In [23]:
print(re.toarray())

[[0.         0.         0.         0.15202883 0.1928293  0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.10062633 0.96414648 0.
  0.        ]
 [0.         0.         0.44120572 0.34785166 0.         0.34785166
  0.34785166 0.         0.         0.44120572 0.         0.
  0.         0.         0.44120572 0.23023945 0.         0.
  0.        ]
 [0.48380259 0.48380259 0.         0.         0.         0.
  0.         0.48380259 0.         0.         0.         0.
  0.48380259 0.         0.         0.25246826 0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.28759007
  0.28759007 0.         0.36477154 0.         0.36477154 0.36477154
  0.         0.36477154 0.         0.19035292 0.         0.36477154
  0.36477154]]


# 高斯模型，鸢尾花分类

In [56]:
# 导入算法包以及数据集
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.naive_bayes import GaussianNB

In [81]:
# 载入数据
iris = datasets.load_iris()
x_train,x_test,y_train,y_test = train_test_split(iris.data, iris.target,random_state=10) 

In [82]:
help(train_test_split)

Help on function train_test_split in module sklearn.model_selection._split:

train_test_split(*arrays, **options)
    Split arrays or matrices into random train and test subsets
    
    Quick utility that wraps input validation and
    ``next(ShuffleSplit().split(X, y))`` and application to input data
    into a single call for splitting (and optionally subsampling) data in a
    oneliner.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.
    
    test_size : float, int, None, optional
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to the
        complement of the train size. By default, the value is set 

In [83]:
iris.data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [84]:
mul_nb = GaussianNB()
mul_nb.fit(x_train,y_train)#训练，拟合

GaussianNB(priors=None)

In [85]:
mul_nb.predict(x_test)#预测

array([1, 2, 0, 1, 0, 1, 1, 1, 0, 1, 1, 2, 1, 0, 0, 2, 1, 0, 0, 0, 2, 2,
       2, 0, 1, 0, 1, 1, 1, 2, 1, 1, 2, 2, 2, 0, 2, 2])

In [86]:
print(classification_report(mul_nb.predict(x_test),y_test))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        11
          1       1.00      1.00      1.00        15
          2       1.00      1.00      1.00        12

avg / total       1.00      1.00      1.00        38



In [87]:
mul_nb.score(x_test,y_test)#模型得分，准确率

1.0

In [55]:
print(confusion_matrix(mul_nb.predict(x_test),y_test))#混淆矩阵

[[12  0  0]
 [ 0 12  1]
 [ 0  1 12]]


# 多项式模型

In [1]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
seed=10
X = np.random.randint(5, size=(6, 10))
y = np.array([1, 2, 3, 4, 5, 6])
clf = MultinomialNB()
clf.fit(X, y)
print(clf.predict(X[2:4]))

[3 4]


In [2]:
X

array([[0, 3, 4, 3, 3, 3, 1, 3, 3, 0],
       [4, 3, 3, 2, 1, 2, 0, 0, 4, 3],
       [4, 4, 2, 0, 4, 3, 3, 0, 4, 1],
       [4, 1, 0, 4, 1, 4, 4, 0, 2, 1],
       [3, 2, 4, 1, 0, 2, 1, 4, 3, 3],
       [1, 2, 2, 1, 1, 1, 0, 0, 3, 4]])

# 伯努利模型

In [22]:
from sklearn.naive_bayes import BernoulliNB
import numpy as np
X = np.random.randint(2, size=(6, 10))
Y = np.array([1, 2, 3, 4, 4, 5])

clf = BernoulliNB()
clf.fit(X, Y)
print(clf.predict(X[2:3]))

[3]


In [23]:
X

array([[0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 1, 0, 1, 1, 0, 1],
       [1, 1, 0, 0, 1, 0, 0, 1, 0, 0],
       [1, 1, 1, 0, 0, 1, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 1, 0, 1, 1, 1, 1, 0]])

# 英文新闻分类，20newsgroups
20 newsgroups数据集18000篇新闻文章，一共涉及到20种话题，所以称作20 newsgroups text dataset，分文两部分：训练集和测试集，通常用来做文本分类.

In [4]:
import os
os.chdir('C:\\Users\\CDA\\data')
from sklearn.datasets import fetch_20newsgroups  #20newsbydate.tar.gz
news = fetch_20newsgroups(data_home='C:\\Users\\CDA\\data',subset='all' )

In [6]:
help(fetch_20newsgroups)

Help on function fetch_20newsgroups in module sklearn.datasets.twenty_newsgroups:

fetch_20newsgroups(data_home=None, subset='train', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True)
    Load the filenames and data from the 20 newsgroups dataset (classification).
    
    Download it if necessary.
    
    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality               1
    Features                  text
    =================   ==========
    
    Read more in the :ref:`User Guide <20newsgroups_dataset>`.
    
    Parameters
    ----------
    data_home : optional, default: None
        Specify a download and cache folder for the datasets. If None,
        all scikit-learn data is stored in '~/scikit_learn_data' subfolders.
    
    subset : 'train' or 'test', 'all', optional
        Select the dataset to load: 'train' for the training set, 'test'
        for the test set, 'all' f

fetch_20newsgroups的参数设置：
fetch_20newsgroups(data_home=None, # 文件下载的路径
                   subset='train', # 加载那一部分数据集 train/test
                   categories=None, # 选取哪一类数据集[类别列表]，默认20类
                   shuffle=True,  # 将数据集随机排序
                   random_state=42, # 随机数生成器
                   remove=(), # ('headers','footers','quotes') 去除部分文本
                   download_if_missing=True # 如果没有下载过，重新下载
                   )

In [7]:
len(news.data)#总的文章书目

18846

In [8]:
news.data[0]#第一篇文章内容

"From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>\nSubject: Pens fans reactions\nOrganization: Post Office, Carnegie Mellon, Pittsburgh, PA\nLines: 12\nNNTP-Posting-Host: po4.andrew.cmu.edu\n\n\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [9]:
news.target_names#新闻的类别

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [10]:
news.target#类别编号

array([10,  3, 17, ...,  3,  1,  7])

In [11]:
news.target.shape#总数,label数量

(18846,)

In [ ]:
#将文本转为TF-IDF向量
from sklearn.feature_extraction.text import TfidfVectorizer
# 提取tfidf特征
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(news.data)
print(vectors.shape)

In [39]:
vectors[1,:]#稀疏矩阵

<1x173762 sparse matrix of type '<class 'numpy.float64'>'
	with 93 stored elements in Compressed Sparse Row format>

In [14]:
#将文本转为TF-IDF向量
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB

tfidf = TfidfVectorizer()
tfidf_data = tfidf.fit_transform(news.data)

nb1 = MultinomialNB()
scores = model_selection.cross_val_score(nb1, tfidf_data, news.target, cv=10)
print(scores.mean())

0.8600671053540412


In [15]:
#添加停用词
def get_stop_words():
    result = set()
    for line in open('stopwords_en.txt','r').readlines():
        result.add(line.strip())
    return result
# 加载停用词
stop_words = get_stop_words()
tfidf = TfidfVectorizer(stop_words=stop_words)
tfidf_data = tfidf.fit_transform(news.data)
nb1 = MultinomialNB()
scores = model_selection.cross_val_score(nb1, tfidf_data, news.target, cv=10)
print(scores.mean())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['camq', 're'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


0.8903168563257069


In [43]:
tfidf_data.shape

(18846, 173489)

In [44]:
# 调整参数
nb1 = MultinomialNB(alpha=0.01)
scores = model_selection.cross_val_score(nb1, tfidf_data, news.target, cv=10)
print(scores.mean())

0.9218343441153822


In [47]:
#和其他方法比较，knn
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors=10)
scores = model_selection.cross_val_score(knn, tfidf_data, news.target, cv=10)
print(scores.mean())

0.7979401356884136


In [20]:
#dtree,决策树
from sklearn import tree
dtree = tree.DecisionTreeClassifier(max_depth=500, min_samples_split=7)
scores = model_selection.cross_val_score(dtree, tfidf_data, news.target, cv=10)
print(scores.mean())

0.6552029420121545
